Industries are increasingly turning to automated chat assistants to handle customer support interactions. But, these tools can only successfully answer questions they were trained on, exposing a growing challenge for enterprise question answering (QA) techniques today. To address this, we are trying to build an intelligent question answering system that doesn’t just return documents related to the question, but extracts relevant information within the documents and puts forth the detailed answer, like one that a human would have come up with. 

In this notebook, we implement a domain-specific question-answering engine that answers any questions related to PyTorch. This model benefits students and working professionals who are beginners in PyTorch. The model is developed with datasets collected from Stack Overflow, PyTorch Github issues, PyTorch documentation, PyTorch discussion forum, and Youtube videos.

### 1- Preliminaries
The implementation presented here relies on the Hugging Face 🤗transformers and 🤗nlp libraries. Wikipedia indexing relies on faiss for the dense version. You can get all of these by running:

<!-- pip install elasticsearch -->
pip install faiss_gpu
pip install nlp
pip install transformers
<!-- 
wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.7.1-linux-x86_64.tar.gz
tar -xzvf elasticsearch-7.7.1-linux-x86_64.tar.gz -->

In [ ]:
! nvidia-smi

Sun Sep 19 03:00:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install faiss_gpu nlp transformers

     |████████████████████████████████| 89.7 MB 9.4 kB/s 
     |████████████████████████████████| 1.7 MB 53.7 MB/s 
     |████████████████████████████████| 2.8 MB 56.9 MB/s 
     |████████████████████████████████| 243 kB 49.0 MB/s 
     |████████████████████████████████| 636 kB 57.4 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import functools
import math
import os  # noqa: F401
from random import choice, randint
from time import time

import numpy as np
import torch
import torch.utils.checkpoint as checkpoint
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm

import faiss  # noqa: F401
import nlp  # noqa: F401
import pandas as pd
from transformers import AdamW, AutoModel, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/TSAI/Capstone_1/data')

In [ ]:
# from lfqa_utils import *

In [ ]:
import os
print(os.getcwd())
path = '/content/drive/MyDrive/TSAI/Capstone_1/data'
os.chdir(path)

/content


In [ ]:
folder = "retriever_models"
# os.chdir(path)
print("current dir is: %s" % (os.getcwd()))

if os.path.isdir(folder):
    print("retriever_models directory exists")
else:
    print("retriever_models directory Doesn't exists, creating one")
    os.mkdir(folder)

current dir is: /content/drive/MyDrive/TSAI/Capstone_1/data
retriever_models directory Doesn't exists, creating one


In [ ]:
folder = "seq2seq_models"
# os.chdir(path)
print("current dir is: %s" % (os.getcwd()))

if os.path.isdir(folder):
    print("seq2seq_models directory exists")
else:
    print("seq2seq_models directory Doesn't exists, creating one")
    os.mkdir(folder)

current dir is: /content/drive/MyDrive/TSAI/Capstone_1/data
seq2seq_models directory Doesn't exists, creating one


## Data Set Preparation
One of the major tasks after collecting data is to clean the datasets and bring all five sets into a common json format.

An example from the final question-answer pair training data set looks like follows :

- Example :

- - *x: Question: Why is PyTorch better than Keras?*
  - *z: Answer Document: In my opinion, this is a very personal question. Every developer/person would have his/her comfort level while deciding on which platform suits them. I will share my opinion. I think PyTorch is better because it is pythonic, (literally like python and you do not need to learn anything new), has higher performance, strong community support, most new papers are written using PyTorch, and has dynamic computation graphs. But again, I might be biased towards it, and maybe you may like something that is better in Keras than Pytorch and might decide to go ahead with it.*
  - *y: Exact Answer: PyTorch is better because it is pythonic, has higher performance,* has*strong community support, most new papers are written using PyTorch, and has dynamic computation graphs.*

We will save all the supporting documents (answer documents) from train and test sets separately in another json for the ease of training. It's because one question might have been addressed in other supporting documents as well (other than the oblivious answer document ). Saving all supporting documents separately benefits us to take advantage of all relevant documents to answer that particular question (We'll see how this works in the following sections).

In [ ]:
with open(path+'/train_data.json') as f:
        train = json.load(f)
with open(path+'/test_data.json') as f:
        test = json.load(f)
with open(path+'/context_master.json') as f:
        passage_snippets = json.load(f)

In [ ]:
train[0]

{'id': 1,
 'x': 'Maxout Layer',
 'y': 'The Maxout layer can be implemented as follows \npython\nclass Maxout(nn.Module):\n def __init__(self, d_in, d_out, pool_size):\n super().__init__()\n self.d_in, self.d_out, self.pool_size = d_in, d_out, pool_size\n self.lin = nn.Linear(d_in, d_out * pool_size)\n def forward(self, inputs):\n shape = list(inputs.size())\n shape[-1] = self.d_out\n shape.append(self.pool_size)\n max_dim = len(shape) - 1\n out = self.lin(inputs)\n m, i = out.view(*shape).max(max_dim)\n return m\n',
 'z': 'For ones who need Maxout, I changed the above code to make it work. \npython\nclass Maxout(nn.Module):\n def __init__(self, d_in, d_out, pool_size):\n super().__init__()\n self.d_in, self.d_out, self.pool_size = d_in, d_out, pool_size\n self.lin = nn.Linear(d_in, d_out * pool_size)\n def forward(self, inputs):\n shape = list(inputs.size())\n shape[-1] = self.d_out\n shape.append(self.pool_size)\n max_dim = len(shape) - 1\n out = self.lin(inputs)\n m, i = out.view(*sh

In [ ]:
len(train)

9140

In [ ]:
test[100]

{'id': 101,
 'x': 'What do Variable(tensor, requires_grad) return instead of Variables?',
 'y': 'Tensors',
 'z': 'The Variable API has been deprecated: Variables are no longer necessary to use autograd with tensors. Autograd automatically supports Tensors with requires_grad set to True. Below please find a quick guide on what has changed: Variable(tensor) and Variable(tensor, requires_grad) still work as expected, but they return Tensors instead of Variables. var.data is the same thing as tensor.data. Methods such as var.backward(), var.detach(), var.register_hook() now work on tensors with the same method names.'}

In [ ]:
len(test)

2286

## Retrieving Support Documents with an Dense retriever Model

Before generating answers for the given query, our QA system needs to find supporting documents. The Retriever module’s job is to find the best candidate documents by calculating the similarity between query and document vectors. We can use either a sparse retriever or a dense retriever to automatically find relevant document snippets for a question.

The sparse retriever works by finding passages that feature the words from the query. However, it has no way to know *a priori* which of these words are more important in context and seems to struggle with understanding the central theme of the query.

Thankfully, some recent works have taken advantage of advances in pre-trained contextual word representations to solve this problem. Models such as [DPR](https://arxiv.org/abs/2004.04906) or [REALM](https://arxiv.org/abs/2002.08909) for example learn to compute a vector representation of the query, as well as vector representations of passages in such a way that the passages that best answer a question maximize the dot product between the two representations. Retrieval is then reduced to a Maximum Inner Product Search, which can be executed efficiently using systems like [FAISS](https://github.com/facebookresearch/faiss).

These successes are very encouraging for our Open-Domain QA application. However, our setup does not quite meet the requirements of either of these approaches. On the one hand, the [DPR](https://arxiv.org/abs/2004.04906) system is trained using gold passage annotations. Unfortunately, we do not have such annotations for our data set. On the other hand, while [REALM](https://arxiv.org/abs/2002.08909) is trained without passage supervision, it requires a pretty expensive pre-training step with an [Inverse Cloze Task](https://arxiv.org/abs/1906.00300) (100,000 steps with batch size 4096), and the ability to re-compute the embeddings of all passages regularly during training.

To train a similar dense retrieval system at reduced cost without having access to gold passage annotation, we will have to **take advantage of another unique feature of our dataset**, **namely the fact that our answers are quite similar in style to the document snippets we want to index. Our answers are summarized, cleaned versions of corresponding document snippets.** Our hypothesis then is that if we train a system to embed the questions and answers in our dataset in a way that allows us to easily match questions to answers, then using the answer embedder on document snippets should allow us to similarly match questions to supporting evidence from document corpus.

In [ ]:
###############
# retriever training
###############
class DatasetQARetriver(Dataset):
    def __init__(self, examples_array, num_rows, extra_answer_threshold=2, min_answer_length=1, training=True, n_samples=None):
        self.data = examples_array
        self.answer_thres = extra_answer_threshold
        self.min_length = min_answer_length
        self.training = training
        self.n_samples = num_rows if n_samples is None else n_samples
        self.num_rows = num_rows

    def __len__(self):
        return self.n_samples

    def make_example(self, idx):
        example = self.data[idx]
        question = example["x"]
        answer = example["y"]
        return (question, answer)

    def __getitem__(self, idx):
        return self.make_example(idx % self.num_rows)


class RetrievalQAEmbedder(torch.nn.Module):
    def __init__(self, sent_encoder, dim):
        super(RetrievalQAEmbedder, self).__init__()
        self.sent_encoder = sent_encoder
        self.output_dim = 128
        self.project_q = torch.nn.Linear(dim, self.output_dim, bias=False)
        self.project_a = torch.nn.Linear(dim, self.output_dim, bias=False)
        self.ce_loss = torch.nn.CrossEntropyLoss(reduction="mean")

    def embed_sentences_checkpointed(self, input_ids, attention_mask, checkpoint_batch_size=-1):
        # reproduces BERT forward pass with checkpointing
        if checkpoint_batch_size < 0 or input_ids.shape[0] < checkpoint_batch_size:
            return self.sent_encoder(input_ids, attention_mask=attention_mask)[1]
        else:
            # prepare implicit variables
            device = input_ids.device
            input_shape = input_ids.size()
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
            head_mask = [None] * self.sent_encoder.config.num_hidden_layers
            extended_attention_mask: torch.Tensor = self.sent_encoder.get_extended_attention_mask(
                attention_mask, input_shape, device
            )

            # define function for checkpointing
            def partial_encode(*inputs):
                encoder_outputs = self.sent_encoder.encoder(inputs[0], attention_mask=inputs[1], head_mask=head_mask,)
                sequence_output = encoder_outputs[0]
                pooled_output = self.sent_encoder.pooler(sequence_output)
                return pooled_output

            # run embedding layer on everything at once
            embedding_output = self.sent_encoder.embeddings(
                input_ids=input_ids, position_ids=None, token_type_ids=token_type_ids, inputs_embeds=None
            )
            # run encoding and pooling on one mini-batch at a time
            pooled_output_list = []
            for b in range(math.ceil(input_ids.shape[0] / checkpoint_batch_size)):
                b_embedding_output = embedding_output[b * checkpoint_batch_size : (b + 1) * checkpoint_batch_size]
                b_attention_mask = extended_attention_mask[b * checkpoint_batch_size : (b + 1) * checkpoint_batch_size]
                pooled_output = checkpoint.checkpoint(partial_encode, b_embedding_output, b_attention_mask)
                pooled_output_list.append(pooled_output)
            return torch.cat(pooled_output_list, dim=0)

    def embed_questions(self, q_ids, q_mask, checkpoint_batch_size=-1):
        q_reps = self.embed_sentences_checkpointed(q_ids, q_mask, checkpoint_batch_size)
        return self.project_q(q_reps)

    def embed_answers(self, a_ids, a_mask, checkpoint_batch_size=-1):
        a_reps = self.embed_sentences_checkpointed(a_ids, a_mask, checkpoint_batch_size)
        return self.project_a(a_reps)

    def forward(self, q_ids, q_mask, a_ids, a_mask, checkpoint_batch_size=-1):
        device = q_ids.device
        q_reps = self.embed_questions(q_ids, q_mask, checkpoint_batch_size)
        a_reps = self.embed_answers(a_ids, a_mask, checkpoint_batch_size)
        compare_scores = torch.mm(q_reps, a_reps.t())#cosine similarity
        loss_qa = self.ce_loss(compare_scores, torch.arange(compare_scores.shape[1]).to(device))#cross entrophy loss
        loss_aq = self.ce_loss(compare_scores.t(), torch.arange(compare_scores.shape[0]).to(device))
        loss = (loss_qa + loss_aq) / 2
        return loss


def make_qa_retriever_model(model_name="google/bert_uncased_L-8_H-512_A-8", from_file=None, device="cuda"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    bert_model = AutoModel.from_pretrained(model_name).to(device)
    # run bert_model on a dummy batch to get output dimension
    d_ids = torch.LongTensor(
        [[bert_model.config.bos_token_id if bert_model.config.bos_token_id is not None else 1]]
    ).to(device)
    d_mask = torch.LongTensor([[1]]).to(device)
    sent_dim = bert_model(d_ids, attention_mask=d_mask)[1].shape[-1]
    qa_embedder = RetrievalQAEmbedder(bert_model, sent_dim).to(device)
    if from_file is not None:
        param_dict = torch.load(from_file)  # has model weights, optimizer, and scheduler states
        qa_embedder.load_state_dict(param_dict["model"])
    return tokenizer, qa_embedder


def make_qa_retriever_batch(qa_list, tokenizer, max_len=128, device="cuda"):
    q_ls = [q for q, a in qa_list]
    a_ls = [a for q, a in qa_list]
 
    q_toks = tokenizer.batch_encode_plus(q_ls, max_length=max_len, pad_to_max_length=True)
    
    q_ids, q_mask = (torch.LongTensor(q_toks["input_ids"]).to(device),torch.LongTensor(q_toks["attention_mask"]).to(device),)
    # print(len(a_ls))

    a_toks = tokenizer.batch_encode_plus(a_ls, max_length=max_len, pad_to_max_length=True)
    # TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
    # print(a_toks)
    a_ids, a_mask = (
        torch.LongTensor(a_toks["input_ids"]).to(device),
        torch.LongTensor(a_toks["attention_mask"]).to(device),
    )

    return (q_ids, q_mask, a_ids, a_mask)


def train_qa_retriever_epoch(model, dataset, tokenizer, optimizer, scheduler, args, e=0):
    model.train()
    # make iterator
    train_sampler = RandomSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_retriever_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda"
    )
    
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # print(next(iter(data_loader)).shape)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    for step, batch in enumerate(epoch_iterator):
        # print("q_ids",q_ids.shape)
        # print(" q_mask,", q_mask.shape)
        # print("A_id", a_ids.shape)
        q_ids, q_mask, a_ids, a_mask = batch
        pre_loss = model(q_ids, q_mask, a_ids, a_mask, checkpoint_batch_size=args.checkpoint_batch_size)
        loss = pre_loss.sum()
        # optimizer
        loss.backward()
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        # some printing within the epoch
        loc_loss += loss.item()
        loc_steps += 1
        if step % args.print_freq == 0 or step == 1:
            print(
                "{:2d} {:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                    e, step, len(dataset) // args.batch_size, loc_loss / loc_steps, time() - st_time,
                )
            )
            loc_loss = 0
            loc_steps = 0


def evaluate_qa_retriever(model, dataset, tokenizer, args):
    model.eval()
    # make iterator
    eval_sampler = SequentialSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_retriever_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda"
    )
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=eval_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    tot_loss = 0.0
    with torch.no_grad():
        for step, batch in enumerate(epoch_iterator):
            q_ids, q_mask, a_ids, a_mask = batch
            loss = model(q_ids, q_mask, a_ids, a_mask)
            tot_loss += loss.item()
        return tot_loss / (step + 1)


def train_qa_retriever(qar_model, qar_tokenizer, qar_train_dset, qar_valid_dset, qar_args):
    qar_optimizer = AdamW(qar_model.parameters(), lr=qar_args.learning_rate, eps=1e-8)
    qar_scheduler = get_linear_schedule_with_warmup(
        qar_optimizer,
        num_warmup_steps=100,
        num_training_steps=(qar_args.num_epochs + 1) * math.ceil(len(qar_train_dset) / qar_args.batch_size),
    )
    for e in range(qar_args.num_epochs):
        train_qa_retriever_epoch(qar_model, qar_train_dset, qar_tokenizer, qar_optimizer, qar_scheduler, qar_args, e)
        m_save_dict = {
            "model": qar_model.state_dict(),
            "optimizer": qar_optimizer.state_dict(),
            "scheduler": qar_scheduler.state_dict(),
        }
        print("Saving model {}".format(qar_args.model_save_name))
        # torch.save(m_save_dict, "{}_{}.pth".format(qar_args.model_save_name, e))
        eval_loss = evaluate_qa_retriever(qar_model, qar_valid_dset, qar_tokenizer, qar_args)
        print("Evaluation loss epoch {:4d}: {:.3f}".format(e, eval_loss))

In [ ]:
# training arguments
class ArgumentsQAR():
    def __init__(self):
        self.batch_size = 512
        self.max_length = 128
        self.checkpoint_batch_size = 32
        self.print_freq = 100
        self.pretrained_model_name = "google/bert_uncased_L-8_H-768_A-12"
        self.model_save_name = "retriever_model_l-8_h-768_b-512-512"
        self.learning_rate = 2e-4
        self.num_epochs =10

qar_args = ArgumentsQAR()

# prepare torch Dataset objects
qar_train_dset = DatasetQARetriver(train,num_rows=len(train), training=True)
qar_valid_dset = DatasetQARetriver(test,num_rows=len(test), training=False)

# load pre-trained BERT and make model
qar_tokenizer, qar_model = make_qa_retriever_model(
        model_name=qar_args.pretrained_model_name,
        from_file=None,
        device="cuda"
)

# train the model
train_qa_retriever(qar_model, qar_tokenizer, qar_train_dset, qar_valid_dset, qar_args)

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/327M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/bert_uncased_L-8_H-768_A-12 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncat

 0     0 of    17 	 L: 6.387 	 -- 10.484
 0     1 of    17 	 L: 6.375 	 -- 20.875
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    0: 5.888
 1     0 of    17 	 L: 5.996 	 -- 10.389
 1     1 of    17 	 L: 5.982 	 -- 20.901
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    1: 5.510
 2     0 of    17 	 L: 5.566 	 -- 10.400
 2     1 of    17 	 L: 5.530 	 -- 20.779
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    2: 5.087
 3     0 of    17 	 L: 5.060 	 -- 10.392
 3     1 of    17 	 L: 5.112 	 -- 20.781
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    3: 4.851
 4     0 of    17 	 L: 4.527 	 -- 10.381
 4     1 of    17 	 L: 4.361 	 -- 20.765
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    4: 4.590
 5     0 of    17 	 L: 3.759 	 -- 10.395
 5     1 of    17 	 L: 3.641 	 -- 20.770
Saving model retriever_model_l-8_h-768_b-512-512
Evaluation loss epoch    5: 4.770
 6     0 o

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Once the model is trained, it can be used to compute passage embeddings for all document corpus. The make_qa_dense_index method takes advantage of numpy memory-mapping, so embeddings are written directly to disk. Again with a single GPU, computing the full set of passage embeddings should take about1 hour.

In [ ]:

###############
# trained retrieval model usage
###############
def embed_passages_for_retrieval(passages, tokenizer, qa_embedder, max_length=512, device="cuda"):
    a_toks = tokenizer.batch_encode_plus(passages, max_length=max_length, pad_to_max_length=True)
    a_ids, a_mask = (
        torch.LongTensor(a_toks["input_ids"]).to(device),
        torch.LongTensor(a_toks["attention_mask"]).to(device),
    )
    with torch.no_grad():
        a_reps = qa_embedder.embed_answers(a_ids, a_mask).cpu().type(torch.float)
    return a_reps.numpy()
def embed_questions_for_retrieval(q_ls, tokenizer, qa_embedder, device="cuda"):
    q_toks = tokenizer.batch_encode_plus(q_ls, max_length=128, pad_to_max_length=True)
    q_ids, q_mask = (
        torch.LongTensor(q_toks["input_ids"]).to(device),
        torch.LongTensor(q_toks["attention_mask"]).to(device),
    )
    with torch.no_grad():
        q_reps = qa_embedder.embed_questions(q_ids, q_mask).cpu().type(torch.float)
    return q_reps.numpy()
def make_qa_dense_index(
    qa_embedder,
    tokenizer,
    passages_dset,
    batch_size=512,
    max_length=128,
    index_name="kilt_passages_reps.dat",
    dtype="float32",
    device="cuda",
):
    st_time = time()
    fp = np.memmap(index_name, dtype=dtype, mode="w+", shape=(len(passages_dset),128))
    n_batches = math.ceil(len(passages_dset) / batch_size)
    for i in range(n_batches):
        passages = [p["z"] for p in passages_dset[i * batch_size : (i + 1) * batch_size]]
        reps = embed_passages_for_retrieval(passages, tokenizer, qa_embedder, max_length, device)
        fp[i * batch_size : (i + 1) * batch_size] = reps
        if i % 50 == 0:
            print(i, time() - st_time)

In [ ]:
# os.chdir(r'/content/drive/MyDrive/TSAI/Capstone_1')

In [ ]:
if not os.path.isfile('passages_reps_32_l-8_h-768_b-512-512.dat'):
  print("hi")

  make_qa_dense_index(
          qar_model, qar_tokenizer, passage_snippets, device='cuda',
          index_name='passages_reps_32_l-8_h-768_b-512-512.dat' )

hi


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0 1.4317922592163086


## Using the Trained Dense Retriever and document Index
Now that we have trained our model to compute query and answer embeddings and used it to compute passage embeddings for all our document snippets, let's see whether it can actually find supporting evidence for a new question. Recalling the the two steps to using the dense retriever: we first compute an embedding for a new question, then do Max Inner Product Search with the pre-computed passage representations.

The MIPS part can be executed efficiently with the faiss library. Additionally, since we computed 128-dimensional passage embeddings, the whole of the representations fits on a GPU, making retrieval even faster. We can create the faiss_gpu index with the following code:

In [ ]:
n_ret = 5

In [ ]:
faiss_res = faiss.StandardGpuResources()
passage_reps = np.memmap(
            'passages_reps_32_l-8_h-768_b-512-512.dat',
            dtype='float32', mode='r',
            # shape=(wiki40b_snippets.num_rows, 128)
            # wiki40b_snippets.num_rows = 11378343,english sections from wiki40B dataset
            shape=(len(passage_snippets), 128)
)

doc_index_flat = faiss.IndexFlatIP(128)
doc_gpu_index = faiss.index_cpu_to_gpu(faiss_res, 0, doc_index_flat)
doc_gpu_index.add(passage_reps)

In [ ]:

def query_qa_dense_index(
    question, qa_embedder, tokenizer, doc_passages, doc_index, n_results=n_ret, min_length=1, device="cuda"
):
    q_rep = embed_questions_for_retrieval([question], tokenizer, qa_embedder, device=device)
    D, I = doc_index.search(q_rep, 2 * n_results)
    res_passages = [doc_passages[int(i)] for i in I[0]]
    support_doc = "<P> " + " <P> ".join([p["z"] for p in res_passages])
    res_list = [dict([(k, p[k]) for k in ["z"]]) for p in res_passages]
    res_list = [res for res in res_list if len(res["z"].split()) > min_length][:n_results]
    for r, sc in zip(res_list, D[0]):
        r["score"] = float(sc)
    return support_doc, res_list

Now we can use the query_qa_dense_index function to query the dense index for our running example question :

In [ ]:
question = test[90]['x']
question

'Parameter not registering if .to(device) is used'

In [ ]:
doc, res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, passage_snippets, doc_gpu_index, device='cuda')
print(res_list)
df = pd.DataFrame({
    
    'Text': ['--- ' + question] + [res['z'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

[{'z': "It's supposed to be `torch.device`", 'score': 27.264524459838867}, {'z': 'your input is on gpu 1 but your net work is on gpu 0.', 'score': 25.201988220214844}, {'z': 'As a data point, in the current C++ API for CUDAStreamGuard, if you pass a stream that lives on a different device, we change *both* the device and the stream.', 'score': 24.98475456237793}, {'z': "DataParallel requires every input tensor be provided on the first device in its device_ids list.\n\nIt basically uses that device as a staging area before scattering to the other GPUs and it's the device where final outputs are gathered before returning from forward. If you want device 2 to be the primary device then you just need to put it at the front of the list as follows\n\nmodel = nn.DataParallel(model, device_ids = [2, 0, 1, 3])\nmodel.to(f'cuda:{model.device_ids[0]}')\n\n\nAfter which all tensors provided to model should be on the first device as well.\n\nx = ... # input tensor\nx = x.to(f'cuda:{model.device_ids

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


,Text
0,--- Parameter not registering if .to(device) is used
1,It's supposed to be `torch.device`
2,your input is on gpu 1 but your net work is on gpu 0.
3,"As a data point, in the current C++ API for CUDAStreamGuard, if you pass a stream that lives on a different device, we change *both* the device and the stream."
4,"DataParallel requires every input tensor be provided on the first device in its device_ids list. It basically uses that device as a staging area before scattering to the other GPUs and it's the device where final outputs are gathered before returning from forward. If you want device 2 to be the primary device then you just need to put it at the front of the list as follows model = nn.DataParallel(model, device_ids = [2, 0, 1, 3]) model.to(f'cuda:{model.device_ids[0]}') After which all tensors provided to model should be on the first device as well. x = ... # input tensor x = x.to(f'cuda:{model.device_ids[0]}') y = model(x)"
5,According to the documentation for torch.cuda.device device (torch.device or int) – device index to select. It’s a no-op if this argument is a negative integer or None. Based on that we could use something like with torch.cuda.device(self.device if self.device.type == 'cuda' else None): # do a bunch of stuff which would simply be a no-op when self.device isn't a CUDA device.


## Retriever Model Evaluation
We have trained a retrieval model that *seems* to be working fine, at least on our running example. Before we use it to answer questions, however, we would like to be able to get some quantitative evaluation of the performance of our dense retriever model.

For the retriever, we want to favor recall over precision as our priority is to make sure that all of the information needed to write the answers is present in the support document. If there is unrelated information, the generation model can learn to sort it out. We measure this by computing the proportion of words in the high-scoring answers which are present in the retrieved support document. To focus on important words, we also weigh answer words by their *Inverse Document Frequency*.This gives us the following IDF-recall scoring function:

In [ ]:

# We first select high-scoring answers (answers beyond the first must have a score of at least 3)
test_qa_list = [(exple['x'],exple['y']) for exple in test]

# We then compute word frequencies in answer text
answer_doc_freq = {}
for q, a in test_qa_list:
    for w in a.lower().split():
        answer_doc_freq[w] = answer_doc_freq.get(w, 0) + 1

# The IDF-recall function is then:
def da_idf_recall(doc, answer):
    d_words = dict([(w, True) for w in doc.lower().split()])
    a_words = answer.lower().split()   
    recall = sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words if w in d_words]) / \
                sum([1. / math.log(1 + answer_doc_freq.get(w, 1)) for w in a_words])
    return recall

In [ ]:
def evaluate_retriever(qa_list, retriever_func, scoring_func, n_ret, verbose=False):
    total_retriever_time = 0.0
    total_retriever_score = 0.0
    st_time = time()
    for i, (question, answer) in enumerate(qa_list):
        r_time = time()
        retrieved_passages = retriever_func(question, n_ret)
        total_retriever_time += time() - r_time
        total_retriever_score += scoring_func(retrieved_passages, answer)
        if verbose and ((i + 1) % 500 == 0 or i <= 1):
            print(
                "{:03d}: S-{:.4f} T-{:.4f} | {:.2f}".format(
                    i + 1, total_retriever_score / (i + 1), total_retriever_time / (i + 1), time() - st_time
                )
            )
    return {"idf_recall": total_retriever_score / (i + 1), "retrieval_time": total_retriever_time / (i + 1)}

In [ ]:
def dense_ret_for_eval(question, n_ret):
    _, dense_res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, passage_snippets, doc_gpu_index, device='cuda')
    dense_doc =" ".join([p["z"] for p in dense_res_list])
    return dense_doc


dense_score = evaluate_retriever(test_qa_list, dense_ret_for_eval, da_idf_recall,n_ret)

df = pd.DataFrame({
    'IDF-Recall': [dense_score['idf_recall']],
    'Time/Query': [dense_score['retrieval_time']],
}, index=[ 'Dense'])
df.style.format({'IDF-Recall': "{:.4f}", 'Time/Query': "{:.4f}"})


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


,IDF-Recall,Time/Query
Sparse,0.2540,0.0093


## Generating Answers with a Sequence-to-Sequence Model

In [ ]:
# generator seq2seq model training
###############
class DatasetS2S(Dataset):
    def __init__(
        self, examples_array,num_rows, make_doc_fun=None, doc_cache=None, training=True
    ):
        self.training = training
        self.data = examples_array
        self.make_doc_function = make_doc_fun
        self.doc_cache = {} if doc_cache is None else doc_cache
        self.num_rows = num_rows
        assert not (make_doc_fun is None and doc_cache is None)
        # make index of specific question-answer pairs from multi-answers
        if self.training:
            self.qa_id_list = [(i, 0) for i in range(self.num_rows)]

        else:
            self.qa_id_list = [(i, 0) for i in range(self.num_rows)]

    def __len__(self):
        return len(self.qa_id_list)

    def make_example(self, idx):
        i, j = self.qa_id_list[idx]
        example = self.data[i]
        question = example["x"] 
        answer = example["y"]
        q_id = example["id"]
        if self.make_doc_function is not None:
            self.doc_cache[q_id] = self.doc_cache.get(q_id, self.make_doc_function(example["x"]))
        document = self.doc_cache[q_id]
        in_st = "question: {} context: {}".format(
            question.lower().strip(), document.lower().strip(),
        )
        out_st = answer
        return (in_st, out_st)

    def __getitem__(self, idx):
        return self.make_example(idx)


def make_qa_s2s_model(model_name="facebook/bart-large", from_file=None, device="cuda"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    if from_file is not None:
        param_dict = torch.load(from_file)  # has model weights, optimizer, and scheduler states
        model.load_state_dict(param_dict["model"])
    return tokenizer, model


def make_qa_s2s_batch(qa_list, tokenizer, max_len=64, max_a_len=128, device="cuda"):
    q_ls = [q for q, a in qa_list]
    a_ls = [a for q, a in qa_list]
    q_toks = tokenizer.batch_encode_plus(q_ls, max_length=max_len, pad_to_max_length=True)
    q_ids, q_mask = (
        torch.LongTensor(q_toks["input_ids"]).to(device),
        torch.LongTensor(q_toks["attention_mask"]).to(device),
    )
    a_toks = tokenizer.batch_encode_plus(a_ls, max_length=min(max_len, max_a_len), pad_to_max_length=True)
    a_ids, a_mask = (
        torch.LongTensor(a_toks["input_ids"]).to(device),
        torch.LongTensor(a_toks["attention_mask"]).to(device),
    )
    labels = a_ids[:, 1:].contiguous().clone()
    labels[a_mask[:, 1:].contiguous() == 0] = -100
    # print(labels)
    model_inputs = {
        "input_ids": q_ids,
        "attention_mask": q_mask,
        "decoder_input_ids": a_ids[:, :-1].contiguous(),
        "labels": labels,
    }
    # print("it'sme",model_inputs)
    return model_inputs


def train_qa_s2s_epoch(model, dataset, tokenizer, optimizer, scheduler, args, e=0, curriculum=True):
    model.train()
    # make iterator
    if curriculum:
        train_sampler = SequentialSampler(dataset)
    else:
        train_sampler = RandomSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda"
    )
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)

  
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    for step, batch_inputs in enumerate(epoch_iterator):
        # print(type(step))
        pre_loss = model(**batch_inputs)[0].unsqueeze(dim=0)
        # print(pre_loss,'pre_loss')
        # print(pre_loss.shape)
        # print(pre_loss.sum(),'sum')
        loss = pre_loss.sum() / pre_loss.shape[0]
        loss.backward()
        # optimizer
        if step % args.backward_freq == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        # some printing within the epoch
        loc_loss += loss.item()
        loc_steps += 1
        if step % args.print_freq == 0 or step == 1:
            print(
                "{:2d} {:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                    e, step, len(dataset) // args.batch_size, loc_loss / loc_steps, time() - st_time,
                )
            )
            loc_loss = 0
            loc_steps = 0


def eval_qa_s2s_epoch(model, dataset, tokenizer, args):
    model.eval()
    # make iterator
    train_sampler = SequentialSampler(dataset)
    model_collate_fn = functools.partial(
        make_qa_s2s_batch, tokenizer=tokenizer, max_len=args.max_length, device="cuda"
    )
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    epoch_iterator = tqdm(data_loader, desc="Iteration", disable=True)
    # accumulate loss since last print
    loc_steps = 0
    loc_loss = 0.0
    st_time = time()
    with torch.no_grad():
        for step, batch_inputs in enumerate(epoch_iterator):
            pre_loss = model(**batch_inputs)[0].unsqueeze(dim=0)
            # print(pre_loss,'pre_loss')
            # print(pre_loss.shape)
            # print(pre_loss.sum(),'sum')
            loss = pre_loss.sum() / pre_loss.shape[0]
            loc_loss += loss.item()
            # print("loc loss here",loc_loss)
            loc_steps += 1
            if step % args.print_freq == 0:
                print(
                    "{:5d} of {:5d} \t L: {:.3f} \t -- {:.3f}".format(
                        step, len(dataset) // args.batch_size, loc_loss / loc_steps, time() - st_time,
                    )
                )
    print("Total \t L: {:.3f} \t -- {:.3f}".format(loc_loss / loc_steps, time() - st_time,))


def train_qa_s2s(qa_s2s_model, qa_s2s_tokenizer, s2s_train_dset, s2s_valid_dset, s2s_args):
    s2s_optimizer = AdamW(qa_s2s_model.parameters(), lr=s2s_args.learning_rate, eps=1e-8)
    s2s_scheduler = get_linear_schedule_with_warmup(
        s2s_optimizer,
        num_warmup_steps=400,
        num_training_steps=(s2s_args.num_epochs + 1) * math.ceil(len(s2s_train_dset) / s2s_args.batch_size),
    )
    for e in range(s2s_args.num_epochs):
        # print((e == 0))

        train_qa_s2s_epoch(
            qa_s2s_model,
            s2s_train_dset,
            qa_s2s_tokenizer,
            s2s_optimizer,
            s2s_scheduler,
            s2s_args,
            e,
            curriculum=True,
        )
        m_save_dict = {
            "model": qa_s2s_model.state_dict(),
            "optimizer": s2s_optimizer.state_dict(),
            "scheduler": s2s_scheduler.state_dict(),
        }
        print("Saving model {}".format(s2s_args.model_save_name))
        eval_qa_s2s_epoch(qa_s2s_model, s2s_valid_dset, qa_s2s_tokenizer, s2s_args)
        # torch.save(m_save_dict, "\{}_{}.pth".format(s2s_args.model_save_name, e))



In [ ]:
n_ret = 2

In [ ]:
# pre-computing support documents
qna_train_docs = []
for example in train:
    support_doc, dense_res_list = query_qa_dense_index(
        example['x'], qar_model, qar_tokenizer,passage_snippets, doc_gpu_index, n_results=n_ret
    )
    qna_train_docs += [(example['id'], support_doc, dense_res_list)]

qna_valid_docs = []
for example in test:
    support_doc, dense_res_list = query_qa_dense_index(
        example['x'], qar_model, qar_tokenizer, passage_snippets, doc_gpu_index, n_results=n_ret
    )
    qna_valid_docs += [(example['id'], support_doc, dense_res_list)]

# training loop proper
class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 2
        self.backward_freq = 16
        self.max_length = 512
        self.print_freq = 100
        self.model_save_name = "s2s_bart_model"
        self.learning_rate = 2e-4
        self.num_epochs = 5

s2s_args = ArgumentsS2S()

# qna_train_docs = json.load(open('precomputed/qna_train_precomputed_dense_docs.json'))
# qna_valid_docs = json.load(open('precomputed/qna_valid_precomputed_dense_docs.json'))
s2s_train_dset = DatasetS2S(train,num_rows =len(train), doc_cache=dict([(k, d) for k, d, src_ls in qna_train_docs]))
s2s_valid_dset = DatasetS2S(test,num_rows =len(test), doc_cache=dict([(k, d) for k, d, src_ls in qna_valid_docs]), training=False)

qa_s2s_tokenizer, pre_model = make_qa_s2s_model(
    model_name="facebook/bart-large",
    from_file=None,
    device="cuda"
)
# qa_s2s_model = torch.nn.DataParallel(pre_model)
qa_s2s_model =pre_model
train_qa_s2s(qa_s2s_model, qa_s2s_tokenizer, s2s_train_dset, s2s_valid_dset, s2s_args)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 0     0 of  4570 	 L: 10.090 	 -- 0.436
 0     1 of  4570 	 L: 19.756 	 -- 0.749
 0   100 of  4570 	 L: 5.388 	 -- 33.072
 0   200 of  4570 	 L: 4.342 	 -- 65.793
 0   300 of  4570 	 L: 4.027 	 -- 98.213
 0   400 of  4570 	 L: 3.936 	 -- 130.459
 0   500 of  4570 	 L: 3.783 	 -- 162.676
 0   600 of  4570 	 L: 3.532 	 -- 195.352
 0   700 of  4570 	 L: 3.236 	 -- 227.447
 0   800 of  4570 	 L: 2.926 	 -- 259.430
 0   900 of  4570 	 L: 3.050 	 -- 291.820
 0  1000 of  4570 	 L: 2.854 	 -- 324.104
 0  1100 of  4570 	 L: 3.297 	 -- 356.097
 0  1200 of  4570 	 L: 3.502 	 -- 387.916
 0  1300 of  4570 	 L: 3.090 	 -- 419.629
 0  1400 of  4570 	 L: 2.913 	 -- 452.050
 0  1500 of  4570 	 L: 2.895 	 -- 484.833
 0  1600 of  4570 	 L: 2.804 	 -- 517.658
 0  1700 of  4570 	 L: 2.789 	 -- 550.460
 0  1800 of  4570 	 L: 3.015 	 -- 583.275
 0  1900 of  4570 	 L: 2.946 	 -- 615.972
 0  2000 of  4570 	 L: 2.554 	 -- 648.777
 0  2100 of  4570 	 L: 2.753 	 -- 681.486
 0  2200 of  4570 	 L: 2.639 	 -- 714.2

In [ ]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.46GB
Available: 22.28GB
Used: 5.57GB
Percentage: 12.5%


In [ ]:
    import torch
    torch.cuda.empty_cache()

We now have everything we need to answer any question! Now let's try the full system on our running example along with the first 100 questions of the test set:

In [ ]:
# generate answer from input "question: ... context: <p> ..."
def qa_s2s_generate(
    question_doc,
    qa_s2s_model,
    qa_s2s_tokenizer,
    num_answers=1,
    num_beams=None,
    min_len=64,
    max_len=512,
    do_sample=False,
    temp=1.0,
    top_p=None,
    top_k=None,
    max_input_length=1024,
    device="cuda:0",
):
    model_inputs = make_qa_s2s_batch([(question_doc, "A")], qa_s2s_tokenizer, max_input_length, device=device,)
    n_beams = num_answers if num_beams is None else max(num_beams, num_answers)
    generated_ids = qa_s2s_model.generate(
        input_ids=model_inputs["input_ids"],
        attention_mask=model_inputs["attention_mask"],
        min_length=min_len,
        max_length=max_len,
        do_sample=do_sample,
        early_stopping=True,
        num_beams=1 if do_sample else n_beams,
        temperature=temp,
        top_k=top_k,
        top_p=top_p,
        eos_token_id=qa_s2s_tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        num_return_sequences=num_answers,
        decoder_start_token_id=qa_s2s_tokenizer.bos_token_id,
    )
    return [qa_s2s_tokenizer.decode(ans_ids, skip_special_tokens=True).strip() for ans_ids in generated_ids]

In [ ]:
questions = []
pred_answers = []
act_answers = []
docs = []

for i in range(100):
    # create support document with the dense index
    question = test[i]['x']
    act_answer = test[i]['y']
    doc, res_list = query_qa_dense_index(
        question, qar_model, qar_tokenizer,
        passage_snippets, doc_gpu_index, device='cuda'
    )
    # concatenate question and support document into BART input
    question_doc = "question: {} context: {}".format(question, doc)
    # generate an answer with beam search
    answer = qa_s2s_generate(
            question_doc, qa_s2s_model, qa_s2s_tokenizer,
            num_answers=1,
            num_beams=8,
            min_len=64,
            max_len=256,
            max_input_length=1024,
            device="cuda:0"
    )[0]
    questions += [question]
    pred_answers += [answer]
    act_answers += [act_answer]
    docs += [doc]

df = pd.DataFrame({
    'Question': questions,
    'Predicted Answer': pred_answers,
    'Actual Answer' :  act_answers,
    'Documents' :docs
})
df.style.set_properties(**{'text-align': 'left'})

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predicted = []
reference = []

# Generate answers for the full test set
for i in range(len(test)):
    # create support document with the dense index
    question = test[i]['x']

    doc, res_list = query_qa_dense_index(
        question, qar_model, qar_tokenizer,
        passage_snippets, doc_gpu_index, device='cuda'
    )
    # concatenate question and support document into BART input
    question_doc = "question: {} context: {}".format(question, doc)
    # generate an answer with beam search
    answer = qa_s2s_generate(
            question_doc, qa_s2s_model, qa_s2s_tokenizer,
            num_answers=1,
            num_beams=8,
            min_len=96,
            max_len=256,
            max_input_length=1024,
            device="cuda:0"
    )[0]
    predicted += [answer]
    reference += [test[i]['y']]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Generator model evaluation 

The last thing we'll do is see how we can get a quantitative evaluation of the model performance. Here, we'll use the ROUGE implementation provided in the nlp library.

In [ ]:
pip install rouge_score rouge_score

In [ ]:
# Compare each generation to the  answer from the dataset
nlp_rouge = nlp.load_metric('rouge')

scores = nlp_rouge.compute(
    predicted, reference,
    rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    use_agregator=True, use_stemmer=False
)
df = pd.DataFrame({
    'rouge1': [scores['rouge1'].mid.precision, scores['rouge1'].mid.recall, scores['rouge1'].mid.fmeasure],
    'rouge2': [scores['rouge2'].mid.precision, scores['rouge2'].mid.recall, scores['rouge2'].mid.fmeasure],
    'rougeL': [scores['rougeL'].mid.precision, scores['rougeL'].mid.recall, scores['rougeL'].mid.fmeasure],
}, index=[ 'P', 'R', 'F'])
df.style.format({'rouge1': "{:.4f}", 'rouge2': "{:.4f}", 'rougeL': "{:.4f}"})

,rouge1,rouge2,rougeL
P,0.1382,0.0231,0.0868
R,0.2680,0.0591,0.2083
F,0.1313,0.0226,0.0864
